# Greedy Algorithms on Trees
###### (Algorithm Design with Haskell)

Given a binary tree, we have a linear time alorithm for producing a tree a minimum height. This is using the bottom-up method.

In [5]:
data Tree a = Leaf a | Node (Tree a) (Tree a) deriving Show

mkTree :: [a] -> Tree a
mkTree = unwrap . until single (pairWith Node) . map Leaf where
  single [_] = True
  single _ = False
  
  pairWith f (x:y:xs) = f x y : pairWith f xs
  pairWith _ xs = xs
  
  unwrap [x] = x

Now we want to produce a tree of natural numbers with minimum cost where cost is defined as

In [6]:
cost :: Tree Int -> Int
cost (Leaf x) = x
cost (Node t1 t2) = 1 + max (cost t1) (cost t2)

The fringe of the tree should be the input list. We can use a greedy algorithm to do this. We start with a specification which is defined using a refinement MinWith rather than an equality
```
mct :: [Int] -> Tree Int
mct xs <- MinWith cost (mkTrees xs)
```

`mkTrees` can be defined in several different ways - it returns all possible ways to form a tree with the given fringe.

In [11]:
mkTrees :: [a] -> [Tree a]
mkTrees [x] = [Leaf x]
mkTrees (x:xs) = concatMap (extend x) $ mkTrees xs

extend :: a -> Tree a -> [Tree a]
extend x (Leaf y) = [Node (Leaf x) (Leaf y)]
extend x (Node t1 t2) =
  Node (Leaf x) (Node t1 t2)
  : [ Node t t2
    | t <- extend x t1
    ]

We can also use a specialized fold function which works over non-empty lists to define `mkTrees` as

In [21]:
foldrn :: (a -> b -> b) -> (a -> b) -> [a] -> b
foldrn f g [x] = g x
foldrn f g (x : xs) = f x $ foldrn f g xs

mkTrees :: [a] -> [Tree a]
mkTrees = foldrn (concatMap . extend) (pure . Leaf)

A second inductive approach to building trees is by starting with a forest and rolling it up

In [22]:
rollup :: [Tree a] -> Tree a
rollup = foldl1 Node

-- and it's dual
spine :: Tree a -> [Tree a]
spine (Leaf x) = [Leaf x]
spine (Node u v) = spine u ++ [v]

mkTrees :: [a] -> [Tree a]
mkTrees = map rollup . mkForests

mkForests :: [a] -> [[Tree a]]
mkForests = foldrn (concatMap . extend) (pure . pure . Leaf) where
  extend x ts = [Leaf x : rollup us : vs
                | k <- [1..length ts], let (us, vs) = splitAt k ts
                ]

Returning to the version of `mkTrees` that is expressed as an instance of `foldrn`, we can use the fusion law of `foldrn` to fuse the two components. The law states that
```
foldrn f2 g2 xs <- M (foldrn f1 g1 xs)
```
for all finite, nonempty lists xs, provided
```
g2 x <- M (g1 x)
f2 x (M (foldrn f1 g1 xs)) <- M (f1 x (foldrn f1 g1 xs))
```
For our purposes, `M = MinWith cost`, `f1 = concatMap extend`, `g1 = pure . Leaf`.

Since `Leaf x = MinWith cost [Leaf x]`, we have that `g2 = Leaf`.

For the second fusion condition we need a function `gstep` that satisfies
```
gstep x (MinWith cost (foldrn (concatMap extend) (pure . Leaf))
  <- MinWith cost (concatMap (extend x) (mkTrees xs))
```
This condition is satisfied if we can show that the monotonicity condition holds
```
cost t <= cost t' => cost (gstep x t) <= cost (gstep x t')
```
but this function doesn't exist. The way forward is to change our cost function so that it uses lexical ordering.

In [23]:
lcost :: Tree Int -> [Int]
lcost = reverse . scanl1 op . map cost . spine
  where
    op x y = 1 + (max x y)

With this, we can specify a gstep that can satisfy the monotonicty condition
```
gstep x ts <- MinWith lcost (extend x ts)
```

In [26]:
gstep :: Int -> Tree Int -> Tree Int
gstep x = rollup . add x . spine

add :: Int -> [Tree Int] -> [Tree Int]
add x ts = Leaf x : join x ts
  where
    join x [u] = [u]
    join x (u:v:ts)
      | x `max` cost u < cost v
      = u:v:ts
      | otherwise
      = join x (Node u v : ts)

However instead of building the spine at each step and then rolling it up, we can roll up the forest at the end of the computation. For this we need `hstep` and `g` for which
```
foldrn gstep Leaf = rollup . foldrn hstep g
```
We arrive at these definition by applying to the fusion law of `foldrn`, this time in the fission direction.

In [27]:
g :: a -> [Tree a]
g = pure . Leaf

hstep = add

We can now produce the final algorithm, which uses tupling to avoid repeated calls to `cost`

In [35]:
type Pair = (Tree Int, Int)

join :: Int -> [Pair] -> [Pair]
join x [u] = [u]
join x (u:v:ts)
  | x `max` snd u < snd v
  = u:v:ts
  | otherwise
  = join x (node u v : ts)

hstep :: Int -> [Pair] -> [Pair]
hstep x ts = leaf x : join x ts

leaf :: Int -> Pair
leaf x = (Leaf x, x)

node :: Pair -> Pair -> Pair
node (t1, c1) (t2, c2) = (Node t1 t2, 1 + max c1 c2)

mct :: [Int] -> Tree Int
mct = rollup . map fst . foldrn hstep (pure . leaf)